In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
#from google.colab import files
#uploaded = files.upload()


In [3]:
import io
df=pd.read_csv('uber.csv')
df.shape

(1156, 7)

In [4]:
df.head()

,START_DATE*,END_DATE*,CATEGORY*,START*,STOP*,MILES*,PURPOSE*
0,1/1/2016 21:11,1/1/2016 21:17,Business,Fort Pierce,Fort Pierce,5.1,Meal/Entertain
1,1/2/2016 1:25,1/2/2016 1:37,Business,Fort Pierce,Fort Pierce,5.0,NaN
2,1/2/2016 20:25,1/2/2016 20:38,Business,Fort Pierce,Fort Pierce,4.8,Errand/Supplies
3,1/5/2016 17:31,1/5/2016 17:45,Business,Fort Pierce,Fort Pierce,4.7,Meeting
4,1/6/2016 14:42,1/6/2016 15:49,Business,Fort Pierce,West Palm Beach,63.7,Customer Visit


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1156 entries, 0 to 1155
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   START_DATE*  1156 non-null   object 
 1   END_DATE*    1155 non-null   object 
 2   CATEGORY*    1155 non-null   object 
 3   START*       1155 non-null   object 
 4   STOP*        1155 non-null   object 
 5   MILES*       1156 non-null   float64
 6   PURPOSE*     653 non-null    object 
dtypes: float64(1), object(6)
memory usage: 63.3+ KB


In [6]:
df.isnull().sum()

START_DATE*      0
END_DATE*        1
CATEGORY*        1
START*           1
STOP*            1
MILES*           0
PURPOSE*       503
dtype: int64

In [7]:
df.dropna(axis=0,inplace=True)
df.isnull().sum()

START_DATE*    0
END_DATE*      0
CATEGORY*      0
START*         0
STOP*          0
MILES*         0
PURPOSE*       0
dtype: int64

In [8]:
def haversine (lon_1, lon_2, lat_1, lat_2):
  lon_1, lon_2, lat_1, lat_2=map(np.radians,[lon_1,lon_2,lat_1,lat_2])
  diff_lon=lon_2-lon_1
  diff_lat=lat_2-lat_1
  km=2*6371*np.arcsin(np.sqrt(np.sin(diff_lat/2.0)**2 +
                            np.cos(lat_1) * np.cos(lat_2)* np.sin(diff_lon/2.0)**2))
  return km

In [9]:
df['Distance']=haversine(df['pickup_longitude'],df['dropoff_longitude'],df['pickup_latitude'],df['dropoff_latitude'])


KeyError: 'pickup_longitude'

In [10]:
df['Distance']=df['Distance'].astype(float).round(2)
df.head()

KeyError: 'Distance'

In [ ]:
plt.scatter(df['Distance'],df['fare_amount'])
plt.xlabel("Distance")
plt.ylabel("fare_amount")

In [ ]:
df.drop(df[df['Distance'] > 60].index,inplace=True)
df.drop(df[df['Distance']== 0].index,inplace=True)
df.drop(df[df['fare_amount']== 0].index,inplace=True)
df.drop(df[df['fare_amount']< 0].index,inplace=True)
df.shape

In [ ]:
df.drop(df[(df['fare_amount']>100)& (df['Distance']<1)].index, inplace=True)
df.drop(df[(df['fare_amount']<100)& (df['Distance']>100)].index, inplace=True)
df.shape


In [ ]:
plt.scatter(df['Distance'],df['fare_amount'])
plt.xlabel("Distance")
plt.ylabel("fare_amount")

In [ ]:
df2=pd.DataFrame().assign(fare=df['fare_amount'],Distance=df['Distance'])
df2.info()

In [ ]:
df2.shape

In [ ]:
plt.figure(figsize=[8,4])
sns.distplot(df2['fare'],color='g',hist_kws=dict(edgecolor="black",linewidth=2),bins=30)
plt.title('Target variable distribution')
plt.show()

In [ ]:
plt.scatter(df2['Distance'],df2['fare'])
plt.xlabel("Distance")
plt.ylabel("fare_amount")


In [ ]:
x=df2['fare']
y=df2['Distance']

In [ ]:
X=df2['Distance'].values.reshape(-1,1)
Y=df2['fare'].values.reshape(-1,1)


In [ ]:
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
y_std=std.fit_transform(Y)
x_std=std.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(x_std, y_std, test_size=0.2, random_state=0)


In [ ]:
from sklearn.linear_model import LinearRegression
l_reg =LinearRegression()
l_reg.fit(X_train,y_train)

In [ ]:
y_pred=l_reg.predict(X_test)

In [ ]:
from sklearn import metrics
print('Mean Absolute error:',metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root MEan Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))


In [ ]:
plt.subplot(2 ,2 , 1)
plt.scatter(X_train, y_train, color= 'red')
plt.plot(X_train, l_reg.predict(X_train), color="blue")
plt.title("Fare vs Distance (Training Set)")
plt.ylabel("Fare_amount")
plt.xlabel("Distance")

plt.subplot(2, 2, 2)
plt.scatter(X_train, y_train, color= 'red')
plt.plot(X_train, l_reg.predict(X_train), color="blue")
plt.ylabel("fare_amout")
plt.xlabel("Distance")
plt.title("Fare vs Distance (Test Set)")